# Embeddings (Vektör Temsilleri)

Bu notebook, bns-nlp-engine'in farklı embedding provider'larını kullanarak metin vektörleştirme özelliklerini gösterir.

In [ ]:
import asyncio
import os
import numpy as np
import time
from bnsnlp.embed import OpenAIEmbedder, CohereEmbedder, HuggingFaceEmbedder

## 1. OpenAI Embeddings

OpenAI'ın text-embedding modelleri ile vektör oluşturma.

In [ ]:
# API key kontrolü
openai_key = os.getenv('OPENAI_API_KEY')

if openai_key:
    print("✓ OpenAI API key bulundu")
else:
    print("✗ OPENAI_API_KEY environment variable tanımlı değil")
    print("  export OPENAI_API_KEY='your-key-here'")

In [ ]:
if openai_key:
    # OpenAI embedder oluştur
    openai_embedder = OpenAIEmbedder({
        'api_key': openai_key,
        'model': 'text-embedding-3-small',
        'batch_size': 16
    })
    
    # Tek metin
    text = "Merhaba dünya, bu bir test metnidir."
    result = await openai_embedder.embed(text)
    
    print(f"Model: {result.model}")
    print(f"Embedding boyutu: {result.dimensions}")
    print(f"İlk 10 değer: {result.embeddings[0][:10]}")

In [ ]:
if openai_key:
    # Batch işleme
    texts = [
        "Python programlama dili",
        "Makine öğrenmesi algoritmaları",
        "Doğal dil işleme teknikleri",
        "Veri bilimi ve analitik",
        "Derin öğrenme modelleri"
    ]
    
    start = time.time()
    result = await openai_embedder.embed(texts)
    elapsed = time.time() - start
    
    print(f"Batch işleme:")
    print(f"  Metin sayısı: {len(texts)}")
    print(f"  Embedding sayısı: {len(result.embeddings)}")
    print(f"  Süre: {elapsed:.3f}s")
    print(f"  Metin/saniye: {len(texts)/elapsed:.1f}")

## 2. Cohere Embeddings

Cohere'in multilingual embedding modelleri.

In [ ]:
# API key kontrolü
cohere_key = os.getenv('COHERE_API_KEY')

if cohere_key:
    print("✓ Cohere API key bulundu")
else:
    print("✗ COHERE_API_KEY environment variable tanımlı değil")
    print("  export COHERE_API_KEY='your-key-here'")

In [ ]:
if cohere_key:
    # Cohere embedder oluştur
    cohere_embedder = CohereEmbedder({
        'api_key': cohere_key,
        'model': 'embed-multilingual-v3.0',
        'batch_size': 96
    })
    
    # Tek metin
    text = "Türkçe doğal dil işleme"
    result = await cohere_embedder.embed(text)
    
    print(f"Model: {result.model}")
    print(f"Embedding boyutu: {result.dimensions}")
    print(f"İlk 10 değer: {result.embeddings[0][:10]}")

## 3. HuggingFace Embeddings (Yerel)

Yerel olarak çalışan sentence-transformers modelleri. API key gerektirmez.

In [ ]:
# HuggingFace embedder oluştur
hf_embedder = HuggingFaceEmbedder({
    'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    'use_gpu': False  # CPU kullan
})

# Tek metin
text = "Merhaba dünya, bu bir test metnidir."
result = await hf_embedder.embed(text)

print(f"Model: {result.model}")
print(f"Embedding boyutu: {result.dimensions}")
print(f"İlk 10 değer: {result.embeddings[0][:10]}")

In [ ]:
# Batch işleme
texts = [
    "Python programlama dili",
    "Makine öğrenmesi algoritmaları",
    "Doğal dil işleme teknikleri",
    "Veri bilimi ve analitik",
    "Derin öğrenme modelleri"
]

start = time.time()
result = await hf_embedder.embed(texts)
elapsed = time.time() - start

print(f"Batch işleme:")
print(f"  Metin sayısı: {len(texts)}")
print(f"  Embedding sayısı: {len(result.embeddings)}")
print(f"  Süre: {elapsed:.3f}s")
print(f"  Metin/saniye: {len(texts)/elapsed:.1f}")

## 4. GPU Desteği

HuggingFace modelleri GPU ile hızlandırılabilir.

In [ ]:
import torch

# GPU kontrolü
if torch.cuda.is_available():
    print(f"✓ GPU bulundu: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA version: {torch.version.cuda}")
    use_gpu = True
else:
    print("✗ GPU bulunamadı, CPU kullanılacak")
    use_gpu = False

In [ ]:
# GPU ile embedder
if use_gpu:
    hf_gpu_embedder = HuggingFaceEmbedder({
        'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
        'use_gpu': True
    })
    
    # Test metinleri
    test_texts = ["Bu bir test metnidir"] * 100
    
    # CPU
    hf_cpu = HuggingFaceEmbedder({'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', 'use_gpu': False})
    start = time.time()
    await hf_cpu.embed(test_texts)
    cpu_time = time.time() - start
    
    # GPU
    start = time.time()
    await hf_gpu_embedder.embed(test_texts)
    gpu_time = time.time() - start
    
    print(f"100 metin için:")
    print(f"  CPU: {cpu_time:.3f}s")
    print(f"  GPU: {gpu_time:.3f}s")
    print(f"  Hızlanma: {cpu_time/gpu_time:.2f}x")

## 5. Embedding Karşılaştırması

Farklı provider'ların embedding'lerini karşılaştırma.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Test metinleri
text1 = "Python programlama dili"
text2 = "Python kodlama"
text3 = "Makine öğrenmesi"

# HuggingFace ile embedding'ler
result = await hf_embedder.embed([text1, text2, text3])
embeddings = np.array(result.embeddings)

# Benzerlik matrisi
similarity = cosine_similarity(embeddings)

print("Cosine Similarity Matrisi:\n")
print(f"{'':25} {text1:25} {text2:25} {text3:25}")
print("-" * 105)
for i, text in enumerate([text1, text2, text3]):
    print(f"{text:25}", end="")
    for j in range(3):
        print(f" {similarity[i][j]:24.4f}", end="")
    print()

## 6. Farklı Modeller Karşılaştırması

In [ ]:
# Farklı HuggingFace modelleri
models = [
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    'sentence-transformers/distiluse-base-multilingual-cased-v2'
]

test_text = "Türkçe doğal dil işleme"

print(f"Test metni: {test_text}\n")
print(f"{'Model':<60} {'Boyut':<10} {'Süre (ms)'}")
print("-" * 85)

for model_name in models:
    embedder = HuggingFaceEmbedder({
        'model': model_name,
        'use_gpu': False
    })
    
    start = time.time()
    result = await embedder.embed(test_text)
    elapsed = (time.time() - start) * 1000
    
    model_short = model_name.split('/')[-1]
    print(f"{model_short:<60} {result.dimensions:<10} {elapsed:.1f}")

## 7. Preprocessing + Embedding Pipeline

In [ ]:
from bnsnlp.preprocess import TurkishPreprocessor

# Preprocessing
preprocessor = TurkishPreprocessor({
    'lowercase': True,
    'remove_punctuation': True,
    'remove_stopwords': True,
    'lemmatize': True
})

# Orijinal metinler
raw_texts = [
    "Python PROGRAMLAMA dili çok güçlüdür!",
    "Makine öğrenmesi yapay zeka'nın alt dalıdır.",
    "Doğal dil işleme, metinleri analiz eder."
]

print("Orijinal metinler:")
for i, text in enumerate(raw_texts, 1):
    print(f"{i}. {text}")

# Preprocessing
preprocessed = await preprocessor.process(raw_texts)
processed_texts = [p.text for p in preprocessed]

print("\nİşlenmiş metinler:")
for i, text in enumerate(processed_texts, 1):
    print(f"{i}. {text}")

# Embedding
result = await hf_embedder.embed(processed_texts)

print(f"\nEmbedding'ler oluşturuldu:")
print(f"  Sayı: {len(result.embeddings)}")
print(f"  Boyut: {result.dimensions}")

## 8. Performans Analizi

In [ ]:
import matplotlib.pyplot as plt

# Farklı batch size'lar
batch_sizes = [1, 4, 8, 16, 32, 64]
test_texts = ["Bu bir test metnidir"] * 100

times = []

for batch_size in batch_sizes:
    embedder = HuggingFaceEmbedder({
        'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
        'use_gpu': False
    })
    
    start = time.time()
    # Batch'lere böl ve işle
    for i in range(0, len(test_texts), batch_size):
        batch = test_texts[i:i+batch_size]
        await embedder.embed(batch)
    elapsed = time.time() - start
    times.append(elapsed)
    
    print(f"Batch size {batch_size:2d}: {elapsed:.3f}s ({len(test_texts)/elapsed:.1f} text/s)")

# Grafik
plt.figure(figsize=(10, 6))
plt.plot(batch_sizes, times, marker='o', linewidth=2, markersize=8)
plt.xlabel('Batch Size', fontsize=12)
plt.ylabel('Time (seconds)', fontsize=12)
plt.title('Embedding Performance vs Batch Size (100 texts)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Embedding Kalitesi Testi

In [ ]:
# Benzer ve farklı metinler
test_cases = [
    ("Python programlama", "Python kodlama", "Benzer"),
    ("Makine öğrenmesi", "Yapay zeka", "İlgili"),
    ("Doğal dil işleme", "Metin analizi", "İlgili"),
    ("Python programlama", "Futbol maçı", "Farklı"),
    ("Makine öğrenmesi", "Yemek tarifi", "Farklı")
]

print(f"{'Metin 1':<25} {'Metin 2':<25} {'Beklenen':<15} {'Benzerlik'}")
print("-" * 80)

for text1, text2, expected in test_cases:
    result = await hf_embedder.embed([text1, text2])
    embeddings = np.array(result.embeddings)
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    
    print(f"{text1:<25} {text2:<25} {expected:<15} {similarity:.4f}")

## 10. Gerçek Dünya Örneği: Döküman Benzerliği

In [ ]:
# Örnek dökümanlar
documents = [
    "Python, yüksek seviyeli bir programlama dilidir. Okunabilir sözdizimi ve güçlü kütüphaneleri ile popülerdir.",
    "Makine öğrenmesi, bilgisayarların veriden öğrenmesini sağlayan yapay zeka dalıdır. Tahmin ve sınıflandırma yapar.",
    "Doğal dil işleme, bilgisayarların insan dilini anlamasını sağlar. Metin analizi ve dil modelleri kullanır.",
    "Veri bilimi, veriden anlamlı bilgiler çıkarma sürecidir. İstatistik, programlama ve alan bilgisi gerektirir.",
    "Derin öğrenme, yapay sinir ağları kullanan makine öğrenmesi yöntemidir. Görüntü ve ses işlemede başarılıdır."
]

# Preprocessing
preprocessed = await preprocessor.process(documents)
processed_docs = [p.text for p in preprocessed]

# Embedding
result = await hf_embedder.embed(processed_docs)
embeddings = np.array(result.embeddings)

# Benzerlik matrisi
similarity_matrix = cosine_similarity(embeddings)

# Görselleştirme
plt.figure(figsize=(12, 10))
plt.imshow(similarity_matrix, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Cosine Similarity')
plt.title('Döküman Benzerlik Matrisi', fontsize=14, pad=20)

labels = ['Python', 'ML', 'NLP', 'Data Sci', 'Deep Learn']
plt.xticks(range(len(labels)), labels, rotation=45, ha='right')
plt.yticks(range(len(labels)), labels)

# Değerleri göster
for i in range(len(labels)):
    for j in range(len(labels)):
        text = plt.text(j, i, f'{similarity_matrix[i, j]:.2f}',
                       ha="center", va="center", color="black", fontsize=10)

plt.tight_layout()
plt.show()

# En benzer döküman çiftleri
print("\nEn benzer döküman çiftleri:\n")
pairs = []
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        pairs.append((i, j, similarity_matrix[i][j]))

pairs.sort(key=lambda x: x[2], reverse=True)

for i, j, sim in pairs[:3]:
    print(f"Benzerlik: {sim:.4f}")
    print(f"  Döküman {i+1}: {documents[i][:60]}...")
    print(f"  Döküman {j+1}: {documents[j][:60]}...")
    print()

## Sonraki Adımlar

- [Search Notebook](search.ipynb) - Embedding'leri kullanarak semantik arama
- [Quickstart Notebook](quickstart.ipynb) - Hızlı başlangıç
- [API Dokümantasyonu](https://yourusername.github.io/bns-nlp-engine/api/embed/)